# 4 - Convolutional Sentiment Analysis

In this notebook, we will be using a *convolutional neural network* (CNN) to conduct sentiment analysis

## Preparing Data

Here, we'll prepare the data. 

As convolutional layers expect the batch dimension to be first we can tell TorchText to return the data already permuted using the `batch_first = True` argument on the field.

In [ ]:
!pip install torchtext==0.6

In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', 
                  tokenizer_language = 'en_core_web_sm',
                  batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED)) #testing with 0.7 split ratio
#train_data, valid_data = train_data.split(split_ratio=0.5, random_state = random.seed(SEED)) # testing with 0.5 split ratio
#train_data, valid_data = train_data.split(split_ratio=0.2, random_state = random.seed(SEED)) #testing with 0.2 split ratio

Build the vocab and load the pre-trained word embeddings.

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

As before, we create the iterators with varying batch sizes for experimentation (32, 64, 128)

In [ ]:
BATCH_SIZE = 64
# BATCH_SIZE = 32
# BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Build the Model

Now to build our model.

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.embedding(text)

        embedded = embedded.unsqueeze(1)
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

We create an instance of our `CNN` class. 

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5 # test with 0.6, and 0.8
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

Next, we'll load the pre-trained embeddings

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

Then zero the initial weights of the unknown and padding tokens.

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

Now, we train the model. We initialize the optimizer, loss function (criterion) and place the model and criterion on the GPU (if available)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters()) #test Adam optimizer
# optimizer = optim.SGD(model.parameters(), lr=1e-3) #test SGD optimizer

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

We implement the function to calculate accuracy.

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def getStats(preds, y):

  rounded_preds = torch.round(torch.sigmoid(preds))

  preds_list = rounded_preds.tolist()
  labels = y.tolist()

  tp = 0.0
  tn = 0.0
  fp = 0.0
  fn = 0.0

  for i in range(len(preds_list)):
    if (preds_list[i] == 1 and labels[i] == 1): tp += 1
    if (preds_list[i] == 0 and labels[i] == 0): tn += 1
    if (preds_list[i] == 1 and labels[i] == 0): fp += 1
    if (preds_list[i] == 0 and labels[i] == 1): fn += 1

  return tp, tn, fp, fn

Here, we calculate precision, recall and F1 score which are other metrics used to compare performance

In [ ]:
def binary_precision(tp, fp):
  if (tp + fp == 0.0): return 0.0
  return tp/(tp+fp)

In [ ]:
def binary_recall(tp, fn):
  if (tp + fn == 0.0): return 0.0
  return tp/(tp + fn)

In [ ]:
def binary_f1(precision, recall):
  if (precision + recall == 0.0): return 0.0
  return (2*precision*recall)/(precision + recall)

Here, we define a function for training our model.

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_prec = 0
    epoch_recall = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)

        tp, tn, fp, fn = getStats(predictions, batch.label)

        prec = binary_precision(tp, fp)

        recall = binary_recall(tp, fn)

        f1_score = binary_f1(prec, recall)

        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_prec += prec
        epoch_recall += recall
        epoch_f1 += f1_score
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_prec / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

Here, we define a function for testing our model.

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_prec = 0
    epoch_recall = 0
    epoch_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            tp, tn, fp, fn = getStats(predictions, batch.label)

            print(tp, tn, fp, fn)

            prec = binary_precision(tp, fp)

            recall = binary_recall(tp, fn)

            f1_score = binary_f1(prec, recall)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_prec += prec
            epoch_recall += recall
            epoch_f1 += f1_score
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_prec / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

Let's define our function to tell us how long epochs take.

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we train our model.

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_prec, train_recall, train_f1 = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_prec, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Prec: {train_prec*100:.2f}% | Test Recall: {train_recall*100:.2f}% | Test F1 Score: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Valid Prec: {valid_prec*100:.2f}% | Valid Recall: {valid_recall*100:.2f}% | Valid F1 Score: {valid_f1*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc, test_prec, test_recall, test_f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test Prec: {test_prec*100:.2f}% | Test Recall: {test_recall*100:.2f}% | Test F1 Score: {test_f1*100:.2f}%')

## User Input

Here, we can check some user-input sentences

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

An example negative review...

In [ ]:
predict_sentiment(model, "This film is terrible")

An example positive review...

In [ ]:
predict_sentiment(model, "This film is great")